<a href="https://colab.research.google.com/github/MN-21/handwriting-dnn-features/blob/main/Curvature_Orientation_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

“Improved Curvature + Orientation MLP”

In [ ]:
# Install dependencies
!pip install --quiet tensorflow tensorflow-datasets opencv-python tqdm scikit-learn

import cv2
import time
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tqdm import tqdm

# 1. Extract curvature and gradient orientation
def extract_curv_and_orientation(images):
    N = images.shape[0]
    curv_abs   = np.zeros((N,28,28), dtype=np.float32)
    curv_sign  = np.zeros((N,28,28), dtype=np.float32)
    orient_map = np.zeros((N,28,28), dtype=np.float32)
    for i, img in enumerate(tqdm(images, desc="Extracting features")):
        gx  = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=3)
        gy  = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=3)
        fxx = cv2.Sobel(img, cv2.CV_32F, 2, 0, ksize=3)
        fyy = cv2.Sobel(img, cv2.CV_32F, 0, 2, ksize=3)
        fxy = cv2.Sobel(img, cv2.CV_32F, 1, 1, ksize=3)
        num   = fxx * gy**2 - 2*fxy * gx * gy + fyy * gx**2
        denom = (gx**2 + gy**2 + 1e-8)**1.5
        k     = num / (denom + 1e-8)
        k_abs = np.abs(k)
        if k_abs.max() > 0:
            k_abs /= k_abs.max()
        curv_abs[i]  = k_abs
        curv_sign[i] = np.sign(k)
        theta = np.arctan2(gy, gx)
        orient_map[i] = (theta + np.pi) / (2*np.pi)
    return curv_abs, curv_sign, orient_map

# 2. Load & combine EMNIST Letters
(ds_train, ds_test), _ = tfds.load(
    'emnist/letters',
    split=['train','test'],
    as_supervised=True,
    with_info=True
)
imgs, lbls = [], []
for img, lbl in tfds.as_numpy(ds_train.concatenate(ds_test)):
    imgs.append(img.squeeze())
    lbls.append(int(lbl) - 1)
imgs = np.stack(imgs)
lbls = np.array(lbls)

# 3. Stratified 80/20 split
X_train, X_test, y_train, y_test = train_test_split(
    imgs, lbls, train_size=0.8,
    stratify=lbls, random_state=42
)

# 4. Compute features
curv_abs_tr, curv_sign_tr, orient_tr = extract_curv_and_orientation(X_train)
curv_abs_te,  curv_sign_te,  orient_te  = extract_curv_and_orientation(X_test)

# 5. Flatten & stack (3×784 = 2352 dims)
def stack_flatten(a, s, o):
    N = a.shape[0]
    return np.hstack([a.reshape(N,-1), s.reshape(N,-1), o.reshape(N,-1)])
X_train_feat = stack_flatten(curv_abs_tr, curv_sign_tr, orient_tr)
X_test_feat  = stack_flatten(curv_abs_te,  curv_sign_te,  orient_te)


# 7. Build MLP model (input dim = 2352)
model = models.Sequential([
    layers.Input(shape=(2352,)),
    layers.Dense(2048), layers.BatchNormalization(), layers.Activation('relu'), layers.Dropout(0.5),
    layers.Dense(1024), layers.BatchNormalization(), layers.Activation('relu'), layers.Dropout(0.5),
    layers.Dense(512),  layers.BatchNormalization(), layers.Activation('relu'), layers.Dropout(0.4),
    layers.Dense(256),  layers.BatchNormalization(), layers.Activation('relu'), layers.Dropout(0.3),
    layers.Dense(26, activation='softmax')
])

# 8. Compile with a float learning rate so ReduceLROnPlateau can adjust it
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 9. Callbacks (patience=8, up to 100 epochs)
es  = callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1)
rlr = callbacks.ReduceLROnPlateau(  monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6, verbose=1)

# 10. Train
t0 = time.time()
history = model.fit(
    X_train_feat, y_train,
    validation_split=0.1,
    epochs=100,
    batch_size=128,
    callbacks=[es, rlr],
    verbose=2
)
train_time = time.time() - t0

# 11. Evaluate & report
t1 = time.time()
loss, acc  = model.evaluate(X_test_feat, y_test, verbose=2)
infer_time = time.time() - t1

print(f"\nImproved Curvature+Orientation MLP on EMNIST Letters")
print(f"Test Accuracy   : {acc*100:.2f}%")
print(f"Training Time   : {train_time:.1f}s")
print(f"Inference Time  : {infer_time:.1f}s for {X_test_feat.shape[0]} samples")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/letters/incomplete.HV9HNT_3.1.0/emnist-test.tfrecord*...:   0%|    …

Dataset emnist downloaded and prepared to /root/tensorflow_datasets/emnist/letters/3.1.0. Subsequent calls will reuse this data.


Extracting features: 100%|██████████| 20720/20720 [00:02<00:00, 7249.24it/s]


Epoch 1/100
583/583 - 117s - 201ms/step - accuracy: 0.6055 - loss: 1.3087 - val_accuracy: 0.7741 - val_loss: 0.7026 - learning_rate: 1.0000e-03
Epoch 2/100
583/583 - 115s - 197ms/step - accuracy: 0.7755 - loss: 0.7051 - val_accuracy: 0.8393 - val_loss: 0.4979 - learning_rate: 1.0000e-03
Epoch 3/100
583/583 - 141s - 242ms/step - accuracy: 0.8147 - loss: 0.5676 - val_accuracy: 0.8488 - val_loss: 0.4564 - learning_rate: 1.0000e-03
Epoch 4/100
583/583 - 141s - 242ms/step - accuracy: 0.8434 - loss: 0.4786 - val_accuracy: 0.8639 - val_loss: 0.4267 - learning_rate: 1.0000e-03
Epoch 5/100
583/583 - 146s - 251ms/step - accuracy: 0.8604 - loss: 0.4192 - val_accuracy: 0.8697 - val_loss: 0.4048 - learning_rate: 1.0000e-03
Epoch 6/100
583/583 - 139s - 239ms/step - accuracy: 0.8755 - loss: 0.3698 - val_accuracy: 0.8748 - val_loss: 0.3962 - learning_rate: 1.0000e-03
Epoch 7/100
583/583 - 145s - 249ms/step - accuracy: 0.8876 - loss: 0.3283 - val_accuracy: 0.8732 - val_loss: 0.4002 - learning_rate: 1.0

In [ ]:
# Install dependencies
!pip install --quiet tensorflow tensorflow-datasets opencv-python tqdm scikit-learn

import cv2
import time
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, callbacks
from tqdm import tqdm

# 1. Extract curvature and gradient orientation
def extract_curv_and_orientation(images):
    N = images.shape[0]
    curv_abs   = np.zeros((N,28,28), dtype=np.float32)
    curv_sign  = np.zeros((N,28,28), dtype=np.float32)
    orient_map = np.zeros((N,28,28), dtype=np.float32)
    for i, img in enumerate(tqdm(images, desc="Extracting features")):
        gx  = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=3)
        gy  = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=3)
        fxx = cv2.Sobel(img, cv2.CV_32F, 2, 0, ksize=3)
        fyy = cv2.Sobel(img, cv2.CV_32F, 0, 2, ksize=3)
        fxy = cv2.Sobel(img, cv2.CV_32F, 1, 1, ksize=3)
        num   = fxx * gy**2 - 2*fxy * gx * gy + fyy * gx**2
        denom = (gx**2 + gy**2 + 1e-8)**1.5
        k     = num / (denom + 1e-8)
        k_abs = np.abs(k)
        if k_abs.max() > 0:
            k_abs /= k_abs.max()
        curv_abs[i]  = k_abs
        curv_sign[i] = np.sign(k)
        theta = np.arctan2(gy, gx)
        orient_map[i] = (theta + np.pi) / (2*np.pi)
    return curv_abs, curv_sign, orient_map

# 2. Load & combine MNIST via TFDS
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],        # train/test splits :contentReference[oaicite:0]{index=0}
    as_supervised=True,
    with_info=True
)
imgs, lbls = [], []
for img, lbl in tfds.as_numpy(ds_train.concatenate(ds_test)):
    imgs.append(img.squeeze())      # shape (28,28)
    lbls.append(int(lbl))           # labels 0–9
imgs = np.stack(imgs)               # (70000,28,28)
lbls = np.array(lbls, dtype=int)

# 3. Stratified 80/20 train/test split
X_train, X_test, y_train, y_test = train_test_split(
    imgs, lbls,
    train_size=0.8,
    stratify=lbls,                  # balanced classes :contentReference[oaicite:1]{index=1}
    random_state=42
)

# 4. Compute features
curv_abs_tr, curv_sign_tr, orient_tr = extract_curv_and_orientation(X_train)
curv_abs_te,  curv_sign_te,  orient_te  = extract_curv_and_orientation(X_test)

# 5. Flatten & stack (3×784 = 2352 dims)
def stack_flatten(a, s, o):
    N = a.shape[0]
    return np.hstack([a.reshape(N,-1), s.reshape(N,-1), o.reshape(N,-1)])
X_train_feat = stack_flatten(curv_abs_tr, curv_sign_tr, orient_tr)
X_test_feat  = stack_flatten(curv_abs_te,  curv_sign_te,  orient_te)

# 6. Standardize features
#scaler = StandardScaler()
#X_train_feat = scaler.fit_transform(X_train_feat)
#X_test_feat  = scaler.transform(X_test_feat)

# 7. Build MLP model (input dim = 2352, output = 10 classes)
model = models.Sequential([
    layers.Input(shape=(2352,)),
    layers.Dense(2048), layers.BatchNormalization(), layers.Activation('relu'), layers.Dropout(0.5),
    layers.Dense(1024), layers.BatchNormalization(), layers.Activation('relu'), layers.Dropout(0.5),
    layers.Dense(512),  layers.BatchNormalization(), layers.Activation('relu'), layers.Dropout(0.4),
    layers.Dense(256),  layers.BatchNormalization(), layers.Activation('relu'), layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

# 8. Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 9. Callbacks
es  = callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1)
rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6, verbose=1)

# 10. Train
t0 = time.time()
history = model.fit(
    X_train_feat, y_train,
    validation_split=0.1,
    epochs=100,
    batch_size=128,
    callbacks=[es, rlr],
    verbose=2
)
train_time = time.time() - t0

# 11. Evaluate & report
t1 = time.time()
loss, acc  = model.evaluate(X_test_feat, y_test, verbose=2)
infer_time = time.time() - t1

print(f"\nImproved Curvature+Orientation MLP on MNIST")
print(f"Test Accuracy   : {acc*100:.2f}%")
print(f"Training Time   : {train_time:.1f}s")
print(f"Inference Time  : {infer_time:.1f}s for {X_test_feat.shape[0]} samples")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.XYJR1B_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


Extracting features: 100%|██████████| 14000/14000 [00:02<00:00, 6642.48it/s]


Epoch 1/100
394/394 - 84s - 213ms/step - accuracy: 0.8545 - loss: 0.4619 - val_accuracy: 0.9305 - val_loss: 0.2272 - learning_rate: 1.0000e-03
Epoch 2/100
394/394 - 83s - 212ms/step - accuracy: 0.9417 - loss: 0.1875 - val_accuracy: 0.9395 - val_loss: 0.1984 - learning_rate: 1.0000e-03
Epoch 3/100
394/394 - 79s - 202ms/step - accuracy: 0.9577 - loss: 0.1331 - val_accuracy: 0.9489 - val_loss: 0.1792 - learning_rate: 1.0000e-03
Epoch 4/100
394/394 - 84s - 212ms/step - accuracy: 0.9670 - loss: 0.1045 - val_accuracy: 0.9568 - val_loss: 0.1479 - learning_rate: 1.0000e-03
Epoch 5/100
394/394 - 82s - 208ms/step - accuracy: 0.9733 - loss: 0.0845 - val_accuracy: 0.9602 - val_loss: 0.1426 - learning_rate: 1.0000e-03
Epoch 6/100
394/394 - 82s - 207ms/step - accuracy: 0.9764 - loss: 0.0729 - val_accuracy: 0.9539 - val_loss: 0.1652 - learning_rate: 1.0000e-03
Epoch 7/100
394/394 - 80s - 203ms/step - accuracy: 0.9789 - loss: 0.0656 - val_accuracy: 0.9602 - val_loss: 0.1539 - learning_rate: 1.0000e-03